In [1]:
import requests
import json
import pandas as pd


In [2]:
#Importing API data
url = "https://data.cdc.gov/resource/hk9y-quqm.json"
response = requests.get(url)
response_json = response.json()
print(json.dumps(response_json, indent=4, sort_keys=True))

[
    {
        "age_group": "0-24",
        "condition": "Influenza and pneumonia",
        "condition_group": "Respiratory diseases",
        "data_as_of": "2020-09-07T00:00:00.000",
        "end_week": "2020-09-06T00:00:00.000",
        "icd10_codes": "J09-J18",
        "number_covid19_deaths": "128",
        "start_week": "2020-02-01T00:00:00.000",
        "state": "US"
    },
    {
        "age_group": "25-34",
        "condition": "Influenza and pneumonia",
        "condition_group": "Respiratory diseases",
        "data_as_of": "2020-09-07T00:00:00.000",
        "end_week": "2020-09-06T00:00:00.000",
        "icd10_codes": "J09-J18",
        "number_covid19_deaths": "622",
        "start_week": "2020-02-01T00:00:00.000",
        "state": "US"
    },
    {
        "age_group": "35-44",
        "condition": "Influenza and pneumonia",
        "condition_group": "Respiratory diseases",
        "data_as_of": "2020-09-07T00:00:00.000",
        "end_week": "2020-09-06T00:00:00.000",
  

In [3]:
#Creating readable files and merging 
covid = pd.read_json(url)
covid

,data_as_of,start_week,end_week,state,condition_group,condition,icd10_codes,age_group,number_covid19_deaths,flag
0,2020-09-07T00:00:00.000,2020-02-01T00:00:00.000,2020-09-06T00:00:00.000,US,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,128.0,NaN
1,2020-09-07T00:00:00.000,2020-02-01T00:00:00.000,2020-09-06T00:00:00.000,US,Respiratory diseases,Influenza and pneumonia,J09-J18,25-34,622.0,NaN
2,2020-09-07T00:00:00.000,2020-02-01T00:00:00.000,2020-09-06T00:00:00.000,US,Respiratory diseases,Influenza and pneumonia,J09-J18,35-44,1571.0,NaN
3,2020-09-07T00:00:00.000,2020-02-01T00:00:00.000,2020-09-06T00:00:00.000,US,Respiratory diseases,Influenza and pneumonia,J09-J18,45-54,4363.0,NaN
4,2020-09-07T00:00:00.000,2020-02-01T00:00:00.000,2020-09-06T00:00:00.000,US,Respiratory diseases,Influenza and pneumonia,J09-J18,55-64,10436.0,NaN
...,...,...,...,...,...,...,...,...,...,...
995,2020-09-07T00:00:00.000,2020-02-01T00:00:00.000,2020-09-06T00:00:00.000,AZ,Circulatory diseases,Heart failure,I50,65-74,40.0,NaN
996,2020-09-07T00:00:00.000,2020-02-01T00:00:00.000,2020-09-06T00:00:00.000,AZ,Circulatory diseases,Heart failure,I50,75-84,75.0,NaN
997,2020-09-07T00:00:00.000,2020-02-01T00:00:00.000,2020-09-06T00:00:00.000,AZ,Circulatory diseases,Heart failure,I50,85+,90.0,NaN
998,2020-09-07T00:00:00.000,2020-02-01T00:00:00.000,2020-09-06T00:00:00.000,AZ,Circulatory diseases,Heart failure,I50,Not stated,0.0,NaN


In [4]:
#Dropping unnecessary data columns
covid_complete = covid.drop(['icd10_codes','data_as_of','start_week', 'end_week', 'flag'], axis=1)
covid_complete

,state,condition_group,condition,age_group,number_covid19_deaths
0,US,Respiratory diseases,Influenza and pneumonia,0-24,128.0
1,US,Respiratory diseases,Influenza and pneumonia,25-34,622.0
2,US,Respiratory diseases,Influenza and pneumonia,35-44,1571.0
3,US,Respiratory diseases,Influenza and pneumonia,45-54,4363.0
4,US,Respiratory diseases,Influenza and pneumonia,55-64,10436.0
...,...,...,...,...,...
995,AZ,Circulatory diseases,Heart failure,65-74,40.0
996,AZ,Circulatory diseases,Heart failure,75-84,75.0
997,AZ,Circulatory diseases,Heart failure,85+,90.0
998,AZ,Circulatory diseases,Heart failure,Not stated,0.0
